In [1]:
import sys, os
sys.path.append("dl_tsc")
from dl_tsc.classifiers.fcn import Classifier_FCN
from preprocessing import ConstantLengthDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split

In [2]:
data_path = './data'
X, y = np.load(f"{data_path}/X.npy", allow_pickle=True), np.load(f"{data_path}//y.npy")
y_encoder = sklearn.preprocessing.OneHotEncoder(categories='auto')
y = y_encoder.fit_transform(y.reshape(-1, 1)).toarray()

In [3]:
number_of_classes = y.shape[1]
output_directory=f'{data_path}/models/fcn/outputs'
os.makedirs(output_directory, exist_ok=True)

In [4]:
fcn = Classifier_FCN(output_directory=output_directory,
                   input_shape=(None, 1),
                  nb_classes=number_of_classes, 
                     verbose=True)

2022-12-02 22:33:45.597109: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-02 22:33:45.609737: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-02 22:33:45.611549: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-02 22:33:45.614046: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, None, 128)         1152      
                                                                 
 batch_normalization (BatchN  (None, None, 128)        512       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, None, 128)         0         
                                                                 
 conv1d_1 (Conv1D)           (None, None, 256)         164096    
                                                                 
 batch_normalization_1 (Batc  (None, None, 256)        1024      
 hNormalization)                                             

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X[[not np.isnan(x).any() for x in X]], 
                                                  y[[not np.isnan(x).any() for x in X]], test_size=0.25)

In [6]:
data_generator_train = ConstantLengthDataGenerator(X_train, y_train)
data_generator_val = ConstantLengthDataGenerator(X_val, y_val)

In [ ]:
history = fcn.model.fit(data_generator_train, epochs=40, validation_data=data_generator_val)

Epoch 1/40


2022-12-02 22:34:02.650737: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200


44/44 [==============================] - 29s 573ms/step - loss: 7.0366 - accuracy: 5.0835e-04 - val_loss: 7.0355 - val_accuracy: 2.9297e-04
Epoch 2/40
44/44 [==============================] - 15s 335ms/step - loss: 7.0340 - accuracy: 6.4174e-04 - val_loss: 7.0310 - val_accuracy: 3.9063e-04
Epoch 3/40
44/44 [==============================] - 17s 380ms/step - loss: 7.0339 - accuracy: 5.7768e-04 - val_loss: 7.0285 - val_accuracy: 4.1118e-04
Epoch 4/40
44/44 [==============================] - 13s 297ms/step - loss: 7.0336 - accuracy: 5.2787e-04 - val_loss: 7.0306 - val_accuracy: 0.0000e+00
Epoch 5/40
44/44 [==============================] - 17s 384ms/step - loss: 7.0340 - accuracy: 5.2267e-04 - val_loss: 7.0340 - val_accuracy: 5.3267e-04
Epoch 6/40
44/44 [==============================] - 13s 284ms/step - loss: 7.0337 - accuracy: 3.6951e-04 - val_loss: 7.0358 - val_accuracy: 6.5104e-04
Epoch 7/40
44/44 [==============================] - 23s 527ms/step - loss: 7.0339 - accuracy: 5.0403e-04 

In [ ]:
figure = plt.figure()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
figure

In [ ]:
figure = plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
figure 

In [ ]:
fcn.model.evaluate(data_generator_val)